In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
driver = webdriver.Chrome(ChromeDriverManager().install())
#%pip install pandas
import pandas as pd
import lxml, html5lib
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
#%pip install openpyxl
import openpyxl




====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/max/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


In [1]:
browser = webdriver.Chrome(executable_path='/Users/max/Desktop/crawler/chromedriver')

NameError: name 'webdriver' is not defined

In [4]:
from selenium.webdriver.chrome.options import Options
#options = Options()
#options.add_argument("--disable-notifications")  ## 擋掉彈跳視窗

browser.get("https://www.ptt.cc/bbs/Gossiping/search?q=%E7%96%AB%E8%8B%97") ##找到要爬的網頁

#search.clear()
search = browser.find_element_by_name("query") ##找到「搜尋」那格的欄位
search.send_keys("莫德納") ##輸入你要的關鍵字「高端 國產疫苗/ bnt 輝瑞/ az/ moderna 莫德納」

button = browser.find_elements_by_xpath("//button[@type = 'submit' ]")[0] ##找到「搜尋」的按紐
button.click()
time.sleep(2)

article_href=[]
for i in range(1,13):  ##這是捲軸往下捲的次數，看你有多少文章要載入，來決定要捲幾次
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight);') ##javascript 指令，用selenium控制捲軸
    soup = BeautifulSoup(browser.page_source)
    results = soup.select('h2.tgn9uw-2.jWUdzO')
    for item in results:
        try:
            item_href = item.select_one("a").get("href")
            article_href.append(item_href.split('/')[4])
        except:
            continue;
    time.sleep(5)

In [23]:
len(article_href)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ptt.cc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ptt.cc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ptt.cc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.htm

In [25]:
import requests
post_id=[]
title =[]
time_post = []
content_text=[]

for i in range(len(article_href)):
    id_ = article_href[i]
    url = 'https://www.dcard.tw/_api/posts/'+id_ ##dcard有給api，就把上面抓下來的id貼上去就好
    requ = requests.get(url)
    post_id.append(requ.json()['id'])
    title.append(requ.json()['title'])
    time_post.append(requ.json()['createdAt'])
    content_text.append(requ.json()['content'])
    

In [30]:
import pandas as pd
D= {'id': post_id, 'title':title,
            'time': time_post, 'content':content_text}

content_pd = pd.DataFrame(D)
content_pd

['/bbs/Gossiping/M.1620007795.A.7E3.html',
 '/bbs/Gossiping/M.1620007396.A.7F0.html',
 '/bbs/Gossiping/M.1620007167.A.B65.html',
 '/bbs/Gossiping/M.1620006662.A.0DF.html',
 '/bbs/Gossiping/M.1620006630.A.68A.html',
 '/bbs/Gossiping/M.1620006608.A.54E.html',
 '/bbs/Gossiping/M.1620005903.A.BD2.html',
 '/bbs/Gossiping/M.1620005860.A.159.html',
 '/bbs/Gossiping/M.1620002568.A.840.html',
 '/bbs/Gossiping/M.1620002266.A.AAA.html',
 '/bbs/Gossiping/M.1619968116.A.7EF.html',
 '/bbs/Gossiping/M.1619946107.A.2C8.html',
 '/bbs/Gossiping/M.1619945862.A.605.html',
 '/bbs/Gossiping/M.1619942986.A.14A.html',
 '/bbs/Gossiping/M.1619940847.A.07E.html',
 '/bbs/Gossiping/M.1619940031.A.889.html',
 '/bbs/Gossiping/M.1619938473.A.ACF.html',
 '/bbs/Gossiping/M.1619932974.A.BF9.html',
 '/bbs/Gossiping/M.1619932881.A.E00.html',
 '/bbs/Gossiping/M.1619930719.A.573.html']

In [2]:
content_pd.to_csv("/Users/max/Desktop/Dcard_莫德納.csv", encoding = "utf_8_sig")

NameError: name 'content_pd' is not defined